$$
R_x(\theta) = \begin{pmatrix} cos(\frac{\theta}{2}) & -isin(\frac{\theta}{2}) \\ -isin(\frac{\theta}{2}) & cos(\frac{\theta}{2}) \end{pmatrix} \\
R_y(\theta) = \begin{pmatrix} cos(\frac{\theta}{2}) & -sin(\frac{\theta}{2}) \\ sin(\frac{\theta}{2}) & cos(\frac{\theta}{2}) \end{pmatrix} \\
R_z(\theta) = \begin{pmatrix} e^{-i\frac{\theta}{2}} & 0 \\ 0 & e^{i\frac{\theta}{2}} \end{pmatrix} \\
$$


$$
\begin{align}
S &= \begin{pmatrix} 1 & 0 \\ 0 & i \end{pmatrix} \\
&\propto R_z(\frac{\pi}{2})
\end{align}
$$

$$
\begin{align}
H &= \frac{1}{\sqrt{2}}\begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix} \\
&\propto R_y(\frac{\pi}{2})R_z(\pi) \\
&\propto R_z(\frac{\pi}{2})R_x(\frac{\pi}{2})R_z(\frac{\pi}{2})
\end{align}
$$

$$
\begin{align}
R_y(\theta) &= SR_x(\theta)S^{\dagger}\\
         &= R_z(\frac{\pi}{2})R_x(\theta)R_z(-\frac{\pi}{2})
\end{align}
$$

$$
\begin{align}
X &= HZH\\
  &= R_z(\frac{\pi}{2})R_x(\frac{\pi}{2})R_z(\frac{\pi}{2})ZR_z(\frac{\pi}{2})R_x(\frac{\pi}{2})R_z(\frac{\pi}{2})\\
  &= R_z(\frac{\pi}{2})R_x(\frac{\pi}{2})ZR_z(\pi)R_x(\frac{\pi}{2})R_z(\frac{\pi}{2})\\
\end{align}
$$

Format of circuit input:
<center>
[[name_of_the_gate, [index_1, index_2]]]
</center>
only 2-qubit gates have index_2.

In [56]:
#example
import qiskit
from qiskit.circuit.library import IGate, XGate, YGate, ZGate, HGate, SGate, RXGate, RYGate, RZGate, CXGate, CZGate
import numpy as np
import random

pi = np.pi

I = IGate().to_matrix()
X = XGate().to_matrix()
Y = YGate().to_matrix()
Z = ZGate().to_matrix()
H = ZGate().to_matrix()
S = ZGate().to_matrix()
RX = lambda t:RXGate(t).to_matrix()
RY = lambda t:RYGate(t).to_matrix()
RZ = lambda t:RZGate(t).to_matrix()
CX = CXGate().to_matrix()
CZ = CZGate().to_matrix()

In [66]:
# Helper functions
#ng = no global phase

def get_param(gate):
    return gate.params[0]

def presice_matrix(gates):
    if len(gates) == 0:
        return 1
    return gates[-1].to_matrix().dot(presice_matrix(gates[:-1]))

def remove_global_phase(mat):
    for i in range(len(mat[0])):
        if np.round(mat[0][i], 3) != 0:
            global_phase_angle = np.angle(mat[0][i])
            break
    global_phase = np.e**(global_phase_angle*1j)
    return mat/global_phase

def matrix(gates, precision=3):
    if precision == None:
        return precise_matrix(gates)
    else:
        return np.round(presice_matrix(gates), precision)

def matrix_ng(gates, precision=3):
    if len(gates) == 0:
        return 1
    mat = presice_matrix(gates)
    mat = remove_global_phase(mat)
    if precision == None:
        return mat
    else:
        return np.round(mat, precision)

def equal(gates1, gates2):
    return (matrix(gates1) == matrix(gates2)).all()

def equal_ng(gates1, gates2):
    return (matrix_ng(gates1) == matrix_ng(gates2)).all()

def equal_mat(mat1, mat2):
    return (mat1 == mat2).all()

def equal_mat_ng(mat1, mat2):
    return (remove_global_phase(mat1) == remove_global_phase(mat2)).all()
    
def commute(gates1, gates2):
    return equal(gates1+gates2, gates2+gates1)

def commute_ng(gates1, gates2):
    return equal_ng(gates1+gates2, gates2+gates1)

def commute_mat(mat1, mat2):
    return equal_mat(mat1.dot(mat2), mat2.dot(mat1))

def commute_mat_ng(mat1, mat2):
    return equal_mat_ng(mat1.dot(mat2), mat2.dot(mat1))

def gate_to_str(gate):
    return gate.name if len(gate.params)==0 else f'{gate.name}({np.round(gate.params[0]/pi, 3)}pi)'

def equal_circuit(qc1, qc2):

def equal_ng_circuit(qc1, qc2):

def equal_test(gates1, gates2):
    print(f'{[gate_to_str(g) for g in gates1]} vs {[gate_to_str(g) for g in gates2]}')
    print(f'equal: {equal(gates1, gates2)}')
    print(f'equal up to global phase: {equal_ng(gates1, gates2)}')
    print()

def commute_test(gates1, gates2):
    print(f'{[gate_to_str(g) for g in gates1]} vs {[gate_to_str(g) for g in gates2]}')
    print(f'commute: {commute(gates1, gates2)}')
    print(f'commute up to global phase: {commute_ng(gates1, gates2)}')
    print()

gates1 = [XGate(), ZGate()]
gates2 = [ZGate(), XGate()]
equal_test(gates1, gates2)

gates1 = [XGate()]
gates2 = [RXGate(pi)]
equal_test(gates1, gates2)

gates1 = [HGate()]
gates2 = [RZGate(pi), RYGate(pi/2)]
equal_test(gates1, gates2)

gates1 = [HGate()]
gates2 = [RZGate(pi/2), RXGate(pi/2), RZGate(pi/2)]
equal_test(gates1, gates2)

gates1 = [RYGate(1.2345)]
gates2 = [RZGate(-pi/2), RXGate(1.2345), RZGate(pi/2)]
equal_test(gates1, gates2)

commute_mat_ng(np.kron(I, RZ(5.4321)), CX)

#print(matrix_ng(gates1))
#print(matrix_ng(gates2))

['x', 'z'] vs ['z', 'x']
equal: False
equal up to global phase: True

['x'] vs ['rx(1.0pi)']
equal: False
equal up to global phase: True

['h'] vs ['rz(1.0pi)', 'ry(0.5pi)']
equal: False
equal up to global phase: True

['h'] vs ['rz(0.5pi)', 'rx(0.5pi)', 'rz(0.5pi)']
equal: False
equal up to global phase: True

['ry(0.393pi)'] vs ['rz(-0.5pi)', 'rx(0.393pi)', 'rz(0.5pi)']
equal: True
equal up to global phase: True



True

In [102]:
GATE_SET = {'i', 'h', 'x', 'y', 'z', 'rx', 'ry', 'rz', 'cx', 'cz'}
BASIS_GATE_SET = {'rx', 'rz', 'cz'}

def random_qc(num_q, num_g, gate_set):
    qc = qiskit.QuantumCircuit(num_q)
    for _ in range(num_g):
        g_name = random.choice(list(gate_set))
        if g_name[0] == 'c':
            c_index, t_index = random.sample(range(num_q), 2)
            getattr(qc, g_name)(c_index, t_index)
        elif g_name[0] == 'r':
            index = random.randint(0, num_q-1)
            param = random.uniform(-pi, pi)
            getattr(qc, g_name)(param, index)
        else:
            index = random.randint(0, num_q-1)
            getattr(qc, g_name)(index)
    return qc

def rewrite_qc(qc):
    output_qc = qiskit.QuantumCircuit(qc.num_qubits)
    for gate_info in qc.data:
        gate = gate_info[0]
        g_name = gate.name
        index_list = gate_info[1]
        if len(index_list) == 1:
            index = index_list.pop()
            if g_name == 'i':
                pass
            elif g_name == 'h':
                output_qc.rz(pi/2, index)
                output_qc.rx(pi/2, index)
                output_qc.rz(pi/2, index)
            elif g_name == 'x':
                output_qc.rx(pi, index)
            elif g_name == 'y':
                output_qc.rz(pi/2, index)
                output_qc.rx(pi, index)
                output_qc.rz(-pi/2, index)
            elif g_name == 'z':
                output_qc.rz(pi, index)
            elif g_name == 'rx':
                output_qc.rx(gate.params[0], index)
            elif g_name == 'ry':
                output_qc.rz(pi/2, index)
                output_qc.rx(gate.params[0], index)
                output_qc.rz(-pi/2, index)
            elif g_name == 'rz':
                output_qc.rz(gate.params[0], index)
        else:
            c_index, t_index = index_list
            if g_name == 'cx':
                output_qc.rz(pi/2, t_index)
                output_qc.rx(pi/2, t_index)
                output_qc.rz(pi/2, t_index)
                output_qc.cz(c_index, t_index)
                output_qc.rz(pi/2, t_index)
                output_qc.rx(pi/2, t_index)
                output_qc.rz(pi/2, t_index)
            elif g_name == 'cz':
                output_qc.cz(c_index, t_index)
    return output_qc

def merge_once(qc):
    output_qc = qiskit.QuantumCircuit(qc.num_qubits)
    num_gates = len(qc.data)
    check_list = []
    for i in range(num_gates):
        if i not in check_list:
            gate_info_i = qc.data[i]
            index_list_i = gate_info_i[1]
            gate_i = gate_info_i[0]
            g_name_i = gate_i.name
            if len(index_list_i) == 1:
                index_i = index_list_i[0]
                for j in range(i+1, num_gates):
                    gate_info_j = qc.data[j]
                    index_list_j = gate_info_j[1]
                    if len(index_list_j) == 1 and index_list_j[0] == index_i:
                        break
                gate_j = gate_info_j[0]
                g_name_j = gate_j.name
                if g_name_j == g_name_i:
                    getattr(output_qc, g_name_j)(gate_i.params[0]+gate_j.params[0], index_i)
                    check_list.append(j)
                else:
                    getattr(output_qc, g_name_i)(gate_i.params[0], index_i)
            else:
                getattr(output_qc, g_name_i)(*index_list_i)
    return output_qc

In [103]:
r_qc = random_qc(3, 20, GATE_SET)
r_qc.draw()

┌───┐     ┌───┐┌──────────────┐┌───┐┌───┐┌─────────────┐»
q_0: ─────┤ H ├─────┤ I ├┤ RX(-0.27722) ├┤ I ├┤ Y ├┤ RY(-2.8419) ├»
     ┌────┴───┴────┐├───┤├──────────────┤├───┤└───┘└────┬───┬────┘»
q_1: ┤ RX(0.43865) ├┤ H ├┤ RX(-0.97352) ├┤ Z ├──■───────┤ H ├─────»
     └┬────────────┤├───┤└──────────────┘└───┘  │       ├───┤     »
q_2: ─┤ RX(1.0675) ├┤ H ├───────────────────────■───────┤ H ├─────»
      └────────────┘└───┘                               └───┘     »
«     ┌────────────┐     
«q_0: ┤ RZ(2.8909) ├─────
«     └───┬───┬────┘┌───┐
«q_1: ────┤ H ├─────┤ H ├
«         ├───┤     ├───┤
«q_2: ────┤ X ├─────┤ H ├
«         └───┘     └───┘

In [104]:
qc = rewrite_qc(r_qc)
qc.draw()

┌──────────┐ ┌──────────┐┌──────────┐┌──────────────┐  ┌──────────┐  »
q_0: ──┤ RZ(pi/2) ├─┤ RX(pi/2) ├┤ RZ(pi/2) ├┤ RX(-0.27722) ├──┤ RZ(pi/2) ├──»
     ┌─┴──────────┴┐├──────────┤├──────────┤└─┬──────────┬─┘┌─┴──────────┴─┐»
q_1: ┤ RX(0.43865) ├┤ RZ(pi/2) ├┤ RX(pi/2) ├──┤ RZ(pi/2) ├──┤ RX(-0.97352) ├»
     └┬────────────┤├──────────┤├──────────┤  ├──────────┤  └──────────────┘»
q_2: ─┤ RX(1.0675) ├┤ RZ(pi/2) ├┤ RX(pi/2) ├──┤ RZ(pi/2) ├──────────────────»
      └────────────┘└──────────┘└──────────┘  └──────────┘                  »
«     ┌────────┐┌───────────┐┌──────────┐┌─────────────┐┌───────────┐»
«q_0: ┤ RX(pi) ├┤ RZ(-pi/2) ├┤ RZ(pi/2) ├┤ RX(-2.8419) ├┤ RZ(-pi/2) ├»
«     ├────────┤└───────────┘├──────────┤└─┬──────────┬┘└┬──────────┤»
«q_1: ┤ RZ(pi) ├──────■──────┤ RZ(pi/2) ├──┤ RX(pi/2) ├──┤ RZ(pi/2) ├»
«     └────────┘      │      ├──────────┤  ├──────────┤  ├──────────┤»
«q_2: ────────────────■──────┤ RZ(pi/2) ├──┤ RX(pi/2) ├──┤ RZ(pi/2) ├»
«                            └──────────┘  └──────────┘  └──────────┘»
«     ┌────────────┐                                                »
«q_0: ┤ RZ(2.8909) ├────────────────────────────────────────────────»
«     └┬──────────┬┘┌──────────┐┌──────────┐┌──────────┐┌──────────┐»
«q_1: ─┤ RZ(pi/2) ├─┤ RX(pi/2) ├┤ RZ(pi/2) ├┤ RZ(pi/2) ├┤ RX(pi/2) ├»
«      └┬────────┬┘ ├──────────┤├──────────┤├──────────┤└──────────┘»
«q_2: ──┤ RX(pi) ├──┤ RZ(pi/2) ├┤ RX(pi/2) ├┤ RZ(pi/2) ├────────────»
«       └────────┘  └──────────┘└──────────┘└──────────┘            »
«                 
«q_0: ────────────
«     ┌──────────┐
«q_1: ┤ RZ(pi/2) ├
«     └──────────┘
«q_2: ────────────
«

In [105]:
m_qc = merge_once(qc)
m_qc.draw()

┌──────────┐ ┌──────────┐┌──────────┐┌──────────────┐  ┌──────────┐  »
q_0: ──┤ RZ(pi/2) ├─┤ RX(pi/2) ├┤ RZ(pi/2) ├┤ RX(-0.27722) ├──┤ RZ(pi/2) ├──»
     ┌─┴──────────┴┐├──────────┤├──────────┤└─┬──────────┬─┘┌─┴──────────┴─┐»
q_1: ┤ RX(0.43865) ├┤ RZ(pi/2) ├┤ RX(pi/2) ├──┤ RZ(pi/2) ├──┤ RX(-0.97352) ├»
     └┬────────────┤├──────────┤├──────────┤  └┬────────┬┘  └──────────────┘»
q_2: ─┤ RX(1.0675) ├┤ RZ(pi/2) ├┤ RX(pi/2) ├───┤ RZ(pi) ├───────────────────»
      └────────────┘└──────────┘└──────────┘   └────────┘                   »
«       ┌────────┐ ┌───────┐┌─────────────┐┌────────────┐            »
«q_0: ──┤ RX(pi) ├─┤ RZ(0) ├┤ RX(-2.8419) ├┤ RZ(1.3201) ├────────────»
«     ┌─┴────────┴┐└───────┘└─┬──────────┬┘└─┬────────┬─┘┌──────────┐»
«q_1: ┤ RZ(3pi/2) ├────■──────┤ RX(pi/2) ├───┤ RZ(pi) ├──┤ RX(pi/2) ├»
«     └───────────┘    │      ├──────────┤  ┌┴────────┴┐ └┬────────┬┘»
«q_2: ─────────────────■──────┤ RX(pi/2) ├──┤ RZ(pi/2) ├──┤ RX(pi) ├─»
«                             └──────────┘  └──────────┘  └────────┘ »
«                                       
«q_0: ──────────────────────────────────
«      ┌────────┐ ┌──────────┐          
«q_1: ─┤ RZ(pi) ├─┤ RX(pi/2) ├──────────
«     ┌┴────────┴┐├──────────┤┌────────┐
«q_2: ┤ RZ(pi/2) ├┤ RX(pi/2) ├┤ RZ(pi) ├
«     └──────────┘└──────────┘└────────┘